In [1]:
import pysqlite3
import sys
sys.modules["sqlite3"] = sys.modules.pop("pysqlite3")

In [2]:
#LLM setup
import os

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

#1,500 requests per minute, 2,048 token limit
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

/home/tommy/code/llm/project/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

file_path = "LLM_ discussion.pdf"
loader = PyPDFLoader(file_path)
data = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)
texts = text_splitter.split_documents(data)

In [7]:
from langchain.vectorstores import Chroma
"""
vector_store = Chroma(
    collection_name="Test",
    embedding_function=embeddings,
    persist_directory="./test",  # Where to save data locally, remove if not necessary
)"""
db = Chroma.from_documents(documents=texts, embedding=embeddings)

In [8]:
retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 5}
)

In [24]:
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
"""
prompt = PromptTemplate.from_template(
    "Use this: {context} \ To answer this {question}"
)
"""
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [34]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

"""
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
"""
context = format_docs(retriever.invoke("Power"))
rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)
response = rag_chain.invoke({"input":"What is power?","context": context})
print(response)

Power is a complex concept that involves awareness of history and power distribution.  The powerful often have an incentive to hide things, which can make it difficult to understand how power operates. 



In [ ]:
# Extras

# Prompt chain
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
)

# Generate prompt
prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)
prompt_template.format(adjective="funny", content="chickens")